In [1]:
import pandas as pd

In [2]:
DATA_PATH = '../data/'
DATA_INTERIM_PATH = DATA_PATH + 'interim/'
train = pd.read_csv(DATA_INTERIM_PATH + 'train_c.csv')

In [9]:
import pickle

DATA_EXTERNAL_PATH = DATA_PATH + 'external/'
pickle_in = open(DATA_EXTERNAL_PATH + "tld.pickle","rb")
tlds = pickle.load(pickle_in)

In [76]:
import ast

ast.literal_eval(train['external_links'][0])

{'https://www.nbcnews.com/news/us-news/betsy-devos-overhaul-obama-era-guidance-campus-sex-assault-n799471': 'said',
 'http://www.washingtontimes.com/news/2017/sep/9/rob-ranco-texas-lawyer-says-he-would-be-ok-if-bets/': 'The Washington Times',
 'https://www.redstate.com/brandon_morse/2017/09/09/cowardly-austin-lawyer-wished-sexual-assault-betsy-devos-locked-twitter-account/': 'deleted',
 'https://twitter.com/PrisonPlanet': '@PrisonPlanet',
 'https://twitter.com/hashtag/RobRanco?src=hash': '#RobRanco',
 'https://www.carlsonattorneys.com/team-member/robert-ranco/': 'Team',
 'https://www.change.org/p/texas-bar-association-removal-of-law-license-for-misogynistic-tweet-of-approval-of-sexual-assault-on-woman?recruiter=778517695&amp;utm_source=share_petition&amp;utm_medium=twitter&amp;utm_campaign=share_twitter_responsive': 'petition'}

In [20]:
from tldextract import extract

def external_links_bias(external_links, tlds) :
    
    HP_links = 0
    nonHP_links = 0
    unknown_links = 0

    for url in ast.literal_eval(external_links):
        tld = extract(url)[1]
        try:
            bias = tlds[tld]
            if bias in ['left', 'right']:
                HP_links += 1
            elif bias in ['leftcenter', 'right-center', 'center']:
                nonHP_links += 1
        except:
            unknown_links += 1
            
    return pd.Series([HP_links, nonHP_links, unknown_links])

In [21]:
train[['HP_links_count', 'nonHP_links_count', 'unknown_links_count']] = train.loc[:,'external_links'].apply(external_links_bias, )

In [27]:
for HP in [True, False]:
    df = train[train['hyperpartisan'] == HP]
    df_shape = df.shape[0]
    HP_links = df['HP_links_count'].sum()
    nonHP_links = df['nonHP_links_count'].sum() 
    unknown_links = df['unknown_links_count'].sum()
    avg_links = (HP_links + nonHP_links + unknown_links)/df_shape
    avg_HP_links = HP_links / df_shape
    avg_nonHP_links = nonHP_links / df_shape
    print(HP)
    print('Average external links = {}'.format(avg_links))
    print('Average HP links = {}'.format(avg_HP_links))
    print('Average nonHP links = {}'.format(avg_nonHP_links))

True
Average external links = 3.146135
Average HP links = 0.3095375
Average nonHP links = 0.8426875
False
Average external links = 1.8485775
Average HP links = 0.104895
Average nonHP links = 0.6900025


In [77]:
train[train['hyperpartisan']==False]['HP_links_count'].value_counts()

0     375583
1      16816
2       4103
3       1518
4        703
5        444
6        263
7        185
8        131
9         74
10        37
11        33
12        25
15        11
13        10
14         9
16         8
19         8
18         6
31         6
21         5
23         4
24         3
32         2
20         2
27         2
29         2
44         1
42         1
35         1
25         1
28         1
22         1
54         1
Name: HP_links_count, dtype: int64

In [83]:
parry = train[train['article_text'].str.lower().str.contains('fusion')].reset_index()

In [85]:
parry['domain'].value_counts()

newsline                10273
counterpunch             1757
foxbusiness              1389
abqjournal               1127
motherjones               696
pri                       614
truthdig                  607
apnews                    542
thedailybeast             505
consortiumnews            466
reuters                   437
dailywire                 364
eppc                      334
baptistnews               248
dissentmagazine           177
nbcnews                   157
calwatchdog               155
chicagoreporter           130
factcheck                 119
billmoyers                117
intrepidreport            113
mintpressnews             110
washingtonblade            94
dcclothesline              90
21stcenturywire            85
thegoldwater               85
foreignpolicyjournal       82
poynter                    82
natmonitor                 74
inthesetimes               70
                        ...  
greensboro                  2
eaglerising                 2
libertyfed

In [86]:
parry[parry['domain']=='newsline'].reset_index()['article_text'][0]

'Singapore issues more financial bans related to 1MDB scandal SINGAPORE (Reuters) ? Singapore?s central bank on Tuesday said it had permanently barred Yeo Jiawei, a former wealth manager of Swiss bank BSI involved in breaches related to Malaysia?s 1MDB fund, from managing financial services firms and advisory activities. In July, a Singapore court jailed Yeo for 4-1/2 years for money laundering and cheating in a case linked to investigations into the siphoning of billions of dollars from Malaysian sovereign fund 1MDB. The Monetary Authority of Singapore said it has issued a permanent prohibition order against Yeo, effective Monday, and a three-year ban for former chief executive of financial advisory firm NRA Capital, Kevin Scully. ?NRA had been appointed to perform the valuation of PetroSaudi Oil Services Limited (PSOSL),? MAS said in a press release. ?Mr Scully had failed to ensure that NRA?s valuation of PSOSL was carried out with sufficient care, judgment and objectivity,? the cent